# Inégalités de genre et développement économique 
Camille BALLU, Gaëlle DORMION et Julie NOWACK, 2A ENSAE Paris, Janvier 2022

# Introduction 

Selon l'OCDE, l'Organisme de Coopération et de Développement Economique, "Les économies sont plus résilientes, productives et inclusives lorsqu'elles réduisent les inégalités entre les sexes et soutiennent activement la participation égale des femmes dans toutes les sphères de la vie". L'idée selon laquelle les inégalités de genre freinent le développement tant sur un plan économique qu'humain semble mise en avant ici.

C'est ce qui nous a conduites à étudier le lien entre la situation économique et sociale des femmes et le niveau de développement du pays. Nous avons modélisé le niveau de développement des pays en nous appuyant sur plusieurs indicateurs: le PIB (Produit Intérieur Brut) par tête et l'IDH (Indice de Développement Humain).  

Pour mesurer ce lien, nous avons construit un indicateur multidimensionnel qui évalue l'impact de la condition des femmes sur le développement. En nous inspirant des travaux menés sur le GII (Gender Inequality Index), cet indicateur prend en compte de multiples facettes de la situation des femmes en lien avec leur activité et ressources économiques, leur santé, l'autonomie vis-à-vis de leurs corps, leur éducation, leur représentation au niveau politique, ou encore leur famille. Lorsque cela a été possible, nous avons choisi de comparer la situation des femmes à celle des hommes dans un même pays.

Pour mener ce travail, nous nous appuyons principalement sur les données de l'OCDE et de la Banque mondiale ainsi que des données scrappées. 

Avant de débuter, il est nécessaire d'avoir installé sur son environnement Python les packages sous les versions suivantes: 

- geopandas>=0.10
- numpy>=1.21
- pycountry>=20.7
- pandas>=1.3.4
- PACKAGE JULIE ? 

In [10]:
import numpy as np
import pandas
import pandas as pd
import matplotlib.pyplot as plt
import pycountry 
import urllib
import bs4
from urllib import request
import statsmodels.api as sm

# 1. Travail préalable de récupération des données

## 1.1. Récupération des données sous le format csv

Les données que nous utilisons proviennent de différentes sources. Nous avons téléchargé nos fichiers .csv sur les sites suivants:
- Données de la Banque mondiale (fichier CSV "WDIData.csv" disponible à l'adresse suivante: https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators?fbclid=IwAR2nT7GW8KtjcMaN6VCc3zAuNVPXsGJ8Ew9AvPACwE4703KmMoyiQuRGkc8). Cette base permet de recueillir des données relatives à la situation économique et sociale d'un pays depuis 1960.  
- Données de l'OCDE (fichier CSV "GIDDB2019_05012022164738556.csv" (et renommé "oecd3.csv" dans la suite du projet) disponible à l'adresse suivante: https://stats.oecd.org/viewhtml.aspx?datasetcode=GIDDB2019&lang=fr). Des données relatives à l'égalité homme-femme y sont disponibles pour l'année 2019. 

- GAELLE (A COMPLETER!!!!!!!!!!!!!!!!!!!!!!)

## 1.2. Scrapping des données

on fait le scrapping qu'on exporte dans des excels avec lun nom cohérent

### 1.2.1. Scrapping du taux de fertilité 

### 1.2.2. Scrapping de la législation sur l'avortement 

On scrappe des données relatives à la législation sur l'avortement sur le site Wikipédia ("https://en.wikipedia.org/wiki/Abortion_law").

In [8]:
url_abortion_law = "https://en.wikipedia.org/wiki/Abortion_law"
request_text = request.urlopen(url_abortion_law).read() 
page = bs4.BeautifulSoup(request_text, "lxml")
table_abortion_law = page.find('table', {'class' : 'wikitable sortable collapsible'})
table_body = table_abortion_law.find('tbody')

rows = table_body.find_all('tr')
cols = rows[1].find_all('td')

dico_abortion = dict()
for row in rows:
    cols = row.find_all('td')
    cols = [ele.text.strip() for ele in cols]
    if len(cols) > 0 : 
        dico_abortion[cols[0]] = cols[1:]

In [26]:
data_abortion = pandas.DataFrame.from_dict(dico_abortion,orient='index')

On nettoie ensuite le dataframe. On choisit de créer une indicatrice traduisant la possibilité pour les femmes d'avorter ou non par choix. 

In [27]:
# Fonction qui va servir pour le nettoyage de la base de données
def cleaner_abortion (char: str):
    assert type(char)==str, "The row does not contains str"
    if '-' in char: return np.NaN
    elif '[' in char:
        i=0
        while char[i+1]!= '[':
            i+=1
        return str(char[:i+1])
    else: return str(char)

In [28]:
for row in rows:
    cols = row.find_all('th')
    print(cols)
    if len(cols) > 0 : 
        cols = [ele.get_text(separator=' ').strip().title() for ele in cols]
        columns_country = cols

[<th>Country</th>, <th style="width:70px">Risk to life</th>, <th style="width:70px">Risk to health</th>, <th style="width:70px">Rape</th>, <th style="width:70px">Fetal impairment</th>, <th style="width:70px">Economic or social</th>, <th style="width:70px">On request
</th>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<th>Country</th>, <th style="width:70px">Risk to life</th>, <th style="width:70px">Risk to health</th>, <th style="width:70px">Rape</th>, <th style="width:70px">Fetal impairment</th>, <th style="width:70px">Economic or social</th>, <th style="width:70px">On request
</th>]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[<th>Country</th>, <th style="width:70px">Risk to life</th>, <th style="width:70px">Risk to health</th>, <th style="width:70px">Rape</th>, <th style="width:70px">Fetal impairment</th>, <th style="width:70px">Economic or social</th>, <th style="widt

In [29]:
data_abortion.columns = columns_country[1:]
data_abortion.reset_index(inplace=True)
data_abortion.rename(columns={'index': 'Country Name'}, inplace=True)
data_abortion.head()

,Country Name,Risk To Life,Risk To Health,Rape,Fetal Impairment,Economic Or Social,On Request
0,Abkhazia[32],prohibited,prohibited,prohibited,prohibited,prohibited,prohibited
1,Afghanistan[needs update?],permitted[k],prohibited,prohibited,prohibited[k],prohibited[k],prohibited
2,Albania[16],22 weeks,22 weeks,22 weeks,no limit,22 weeks,12 weeks
3,Algeria[39],permitted,permitted,prohibited,prohibited,prohibited,prohibited
4,Andorra,prohibited[l],prohibited,prohibited,prohibited,prohibited,prohibited


In [30]:
data_abortion2=data_abortion
data_abortion2['abortion'] = data_abortion2['On Request'].str.contains("permitted|weeks", na=False).astype(int)
data_abortion2["Country Name"]=data_abortion2["Country Name"].apply(cleaner_abortion)
data_abortion2=data_abortion2[["Country Name", "abortion"]]
data_abortion2.head(5)

,Country Name,abortion
0,Abkhazia,0
1,Afghanistan,0
2,Albania,1
3,Algeria,0
4,Andorra,0


In [23]:
# Exportation en .csv 
data_abortion2.to_csv('data_abortion2.csv')

## 1.3. Nettoyage des bases de données de la Banque mondiale et de l'OCDE

### 1.3.1. Df de la Banque mondiale

In [32]:
# Dataframe de la Banque mondiale 
df_bm=pd.read_csv("WDIData.csv")
df_bm.head(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,Unnamed: 65
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.320475,16.643243,16.994695,17.313168,17.604950,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,32.224027,32.046478,31.323579,33.312163,38.380433,39.754201,42.168241,43.640661,NaN,NaN
2,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,20.525353,19.461383,17.790698,16.553470,23.907897,24.624725,26.813900,28.841150,NaN,NaN


In [33]:
# On se restreint aux observations de 2019
df_bm2=df_bm[["Country Code", "Country Name", "Indicator Name", "Indicator Code", "2019"]]
df_bm2.head(5)

,Country Code,Country Name,Indicator Name,Indicator Code,2019
0,AFE,Africa Eastern and Southern,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN
1,AFE,Africa Eastern and Southern,Access to electricity (% of population),EG.ELC.ACCS.ZS,43.640661
2,AFE,Africa Eastern and Southern,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,28.841150
3,AFE,Africa Eastern and Southern,"Access to electricity, urban (% of urban popul...",EG.ELC.ACCS.UR.ZS,73.589886
4,AFE,Africa Eastern and Southern,Account ownership at a financial institution o...,FX.OWN.TOTL.ZS,NaN


### 1.3.2. Df de l'OCDE

In [34]:
df_ocde=pd.read_csv('./oecd3.csv', sep=',', encoding='latin-1')
df_ocde.head(5)

,"ï»¿""REGION""",RÃ©gion,LOCATION,Pays,INC,Revenu,VAR,Variable,TIME,AnnÃ©e,Value,Flag Codes,Flags
0,ASI,Asie,AUS,Australie,HIN,ÃlevÃ©,DF_HR_LAW,Loi,2019,2019,0.50,NaN,NaN
1,ASI,Asie,AUS,Australie,HIN,ÃlevÃ©,DF_HR_ATT,Attitudes,2019,2019,21.10,NaN,NaN
2,ASI,Asie,AUS,Australie,HIN,ÃlevÃ©,DF_HR_PRACT,Pratique,2019,2019,1.82,NaN,NaN
3,ASI,Asie,AUS,Australie,HIN,ÃlevÃ©,DF_DV_LAW,Loi,2019,2019,0.00,NaN,NaN
4,ASI,Asie,AUS,Australie,HIN,ÃlevÃ©,DF_IN_LAW,Loi,2019,2019,0.00,NaN,NaN


In [35]:
# on ne sélectionne que les lignes 'pratique' qui correspondent à la réalité des faits 
df_ocde2=df_ocde[df_ocde['Variable']=="Pratique"]
df_ocde2.rename(columns={'ï»¿"REGION"': 'region_monde', 'RÃ©gion': 'region', 'LOCATION': 'Country Code', 'Pays':'Country Name'}, inplace=True)
df_ocde2=df_ocde2[df_ocde2['region_monde']=="ALL"]
df_ocde3=df_ocde2[['Country Code', 'Country Name', 'VAR', 'Value']]
df_ocde3.head(5)

C:\Users\julie\anaconda3\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,Country Code,Country Name,VAR,Value
29,AUS,Australie,DF_HR_PRACT,1.82
34,AUS,Australie,RPI_VAW_PRACT,16.90
36,AUS,Australie,RPI_MW_PRACT,105.50
38,AUS,Australie,RPI_RA_PRACT,11.00
42,AUS,Australie,RAPFR_SAFS_PRACT,50.00


# 2.Sélection des indicateurs pertinents

- stats descriptives sur les indicateurs de la banque mondiale, ocde,... avec sélection des variables women, female,... 
- parler d'indicateurs pertinents et pas pertinents 
- à la fin mettre le tableau des proportionnalités de Julie

# 3. Création de la base de données

- je pense qu'ici il faudrait qu'on ait une base globale avec tous les indicateurs pertinets fusionnés de l'OCDE, banque mondiale,... (il me semble que c'est dans les consignes mais pas sure il faudriat vérifier)

# 4. Création des indicateurs dimensionnels des inégalités hommes/femmes

- au début on met les fonctions qui permettent de sélectionner les variables + de normaliser dans la base globale qu'on a créée + pondéraion
- création de chaque indicateur avec pondérations 

# 5. Création de l'indicateur global 

- indicateur global 
- régression

# 6. Statistiques globales sur l'indicateur

- boxplot 
- graph en bâtons par continents 

# 7. Visualisation de l'indicateur 

- carte

# Conclusion